Schrödinger-like equation for Generalized Pöschl-Teller potential:

$\dfrac{d^2\psi}{dx^2} + \left( \omega^2 - \alpha^2 V_0\text{sech}^2(\alpha x) \right)\psi = 0$

In [418]:
from sympy import *

x,y = symbols("x y", real=True)
w = symbols("\omega")

v = symbols("V_0", real=True)
a = symbols("a", positive=True)
b = symbols("b", positive=True)

psi = Function("\psi")(x)
psi_y = Function("\psi")(y)
phi = Function("\phi")(y)

sch_eq = Eq( diff(diff(psi,x),x) + (w**2-a**2*v*sech(a*x)**2)*psi, 0 )#.subs(v,S(1)/2).subs(a,1)
display(sch_eq)

Eq((-V_0*a**2*sech(a*x)**2 + \omega**2)*\psi(x) + Derivative(\psi(x), (x, 2)), 0)

In [419]:
#Function to change variables (https://stackoverflow.com/questions/57840957/differential-equation-change-of-variables-with-sympy)
def variable_change(ODE,dependent_var, 
                    independent_var,
                    new_dependent_var = None, 
                    new_independent_var= None, 
                    dependent_var_relation = None,
                    independent_var_relation = None,
                    order = 2):

    if new_dependent_var == None:
        new_dependent_var = dependent_var
    if new_independent_var == None:
        new_independent_var = independent_var

    # dependent variable change

    if new_independent_var != independent_var:

        for i in range(order, -1, -1):

            # remplace derivate
            a = diff(dependent_var , independent_var, i )
            ξ = Function("ξ")(independent_var)

            b = diff( dependent_var.subs(independent_var, ξ),  independent_var  ,i)

            rel = solve(independent_var_relation, new_independent_var)[0]

            for j in range(order, 0, -1):
                b = b.subs( diff(ξ,independent_var,j), diff(rel,independent_var,j))

            b = b.subs(ξ, new_independent_var)

            rel = solve(independent_var_relation, independent_var)[0]
            b = b.subs(independent_var, rel)

            ODE =   ODE.subs(a,b)

        ODE = ODE.subs(independent_var, rel)

    # change of variables of indpendent variable

    if new_dependent_var != dependent_var:

        ODE = (ODE.subs(dependent_var.subs(independent_var,new_independent_var) , (solve(dependent_var_relation, dependent_var)[0])))
        ODE = ODE.doit().expand()

    return ODE.simplify()

Change of variables 

$x = \dfrac{1}{\alpha}\tanh(y) \iff y = \text{atanh}(\alpha x)  $

In [420]:
sch_eq_y = variable_change(
    ODE=sch_eq,
    independent_var=x,
    new_independent_var=y,
    independent_var_relation=Eq(x,1/a*atanh(y)),
    dependent_var=psi,
    new_dependent_var=psi,
    dependent_var_relation=None,
    order=2
)

display(sch_eq_y)

Eq(2*a**2*y*(y**2 - 1)*Derivative(\psi(y), y) + a**2*(y**2 - 1)**2*Derivative(\psi(y), (y, 2)) + (V_0*a**2*(y**2 - 1) + \omega**2)*\psi(y), 0)

In [421]:
sch_eq_y.subs(v,S(1)/2).subs(a, 1) # Check limit holds

Eq(2*y*(y**2 - 1)*Derivative(\psi(y), y) + (y**2 - 1)**2*Derivative(\psi(y), (y, 2)) + (\omega**2 + y**2/2 - 1/2)*\psi(y), 0)

Incorporate boundary conditions (second equality simpler)

$ \psi(y) = (1-y)^{(-i\omega/2)}(1+y)^{(-i\omega/2)} = (1-y^2)^{(-i\omega/2)} $

In [422]:
sch_eq_bound = sch_eq_y.subs(psi_y, (1-y**2)**(-I*w/2)*phi).doit()
sch_eq_bound

Eq(2*a**2*y*(y**2 - 1)*(I*\omega*y*\phi(y)/((1 - y**2)*(1 - y**2)**(I*\omega/2)) + Derivative(\phi(y), y)/(1 - y**2)**(I*\omega/2)) + a**2*(y**2 - 1)**2*(-2*I*\omega*y*Derivative(\phi(y), y)/(y**2 - 1) - \omega*(\omega*y**2/(y**2 - 1) - 2*I*y**2/(y**2 - 1) + I)*\phi(y)/(y**2 - 1) + Derivative(\phi(y), (y, 2)))/(1 - y**2)**(I*\omega/2) + (V_0*a**2*(y**2 - 1) + \omega**2)*\phi(y)/(1 - y**2)**(I*\omega/2), 0)

In [423]:
sch_factor = sch_eq_bound.lhs.collect( (1-y**2)**(-I*w/2) )*(1-y**2)**(I*w/2)
sch_factor

2*a**2*y*(y**2 - 1)*(I*\omega*y*\phi(y)/(1 - y**2) + Derivative(\phi(y), y)) + a**2*(y**2 - 1)**2*(-2*I*\omega*y*Derivative(\phi(y), y)/(y**2 - 1) - \omega*(\omega*y**2/(y**2 - 1) - 2*I*y**2/(y**2 - 1) + I)*\phi(y)/(y**2 - 1) + Derivative(\phi(y), (y, 2))) + (V_0*a**2*(y**2 - 1) + \omega**2)*\phi(y)

In [424]:
A,B,C = symbols("A B C") #A,B,C represent second, first and zero-th order derivatives of phi
sch_eq_abc = Eq( sch_factor.subs( diff(diff(phi,y),y), A ).subs( diff(phi,y), B ).subs(phi, C), 0)
#sch_eq_y = factor(sch_eq_y)
display(sch_eq_abc)

Eq(C*(V_0*a**2*(y**2 - 1) + \omega**2) + 2*a**2*y*(B + I*C*\omega*y/(1 - y**2))*(y**2 - 1) + a**2*(y**2 - 1)**2*(A - 2*I*B*\omega*y/(y**2 - 1) - C*\omega*(\omega*y**2/(y**2 - 1) - 2*I*y**2/(y**2 - 1) + I)/(y**2 - 1)), 0)

In [429]:
sch_eq_abc_col = Eq(sch_eq_abc.lhs.expand().collect(A).collect(B).collect(C),0)
sch_eq_abc_col

Eq(A*(a**2*y**4 - 2*a**2*y**2 + a**2) + B*(-2*I*\omega*a**2*y**5/(y**2 - 1) + 4*I*\omega*a**2*y**3/(y**2 - 1) - 2*I*\omega*a**2*y/(y**2 - 1) + 2*a**2*y**3 - 2*a**2*y) + C*(V_0*a**2*y**2 - V_0*a**2 - \omega**2*a**2*y**6/(y**4 - 2*y**2 + 1) + 2*\omega**2*a**2*y**4/(y**4 - 2*y**2 + 1) - \omega**2*a**2*y**2/(y**4 - 2*y**2 + 1) + \omega**2 + 2*I*\omega*a**2*y**6/(y**4 - 2*y**2 + 1) - 4*I*\omega*a**2*y**4/(y**4 - 2*y**2 + 1) - I*\omega*a**2*y**4/(y**2 - 1) + 2*I*\omega*a**2*y**4/(1 - y**2) + 2*I*\omega*a**2*y**2/(y**4 - 2*y**2 + 1) + 2*I*\omega*a**2*y**2/(y**2 - 1) - 2*I*\omega*a**2*y**2/(1 - y**2) - I*\omega*a**2/(y**2 - 1)), 0)

In [436]:
sch_eq_abc_col_sim = sch_eq_abc_col.lhs.simplify().collect(A).collect(B).collect(C)
sch_eq_abc_col_sim

A*(a**2*y**4 - 2*a**2*y**2 + a**2) + B*(-2*I*\omega*a**2*y**3 + 2*I*\omega*a**2*y + 2*a**2*y**3 - 2*a**2*y) + C*(V_0*a**2*y**2 - V_0*a**2 - \omega**2*a**2*y**2 + \omega**2 - I*\omega*a**2*y**2 + I*\omega*a**2)

In [427]:
# Normalization
norm = (a**2*y**4 - 2*a**2*y**2 + a**2)
norm

a**2*y**4 - 2*a**2*y**2 + a**2

In [444]:
sch_new = Eq( A + B * (-2*I*w*a**2*y**3 + 2*I*w*a**2*y + 2*a**2*y**3 - 2*a**2*y) / norm + C * (v*a**2*y**2 - v*a**2 - w**2*a**2*y**2 + w**2 - I*w*a**2*y**2 + I*w*a**2) / norm, 0 )
sch_new

Eq(A + B*(-2*I*\omega*a**2*y**3 + 2*I*\omega*a**2*y + 2*a**2*y**3 - 2*a**2*y)/(a**2*y**4 - 2*a**2*y**2 + a**2) + C*(V_0*a**2*y**2 - V_0*a**2 - \omega**2*a**2*y**2 + \omega**2 - I*\omega*a**2*y**2 + I*\omega*a**2)/(a**2*y**4 - 2*a**2*y**2 + a**2), 0)

In [447]:
((-2*I*w*a**2*y**3 + 2*I*w*a**2*y + 2*a**2*y**3 - 2*a**2*y) / norm).simplify()

2*y*(-I*\omega + 1)/(y**2 - 1)

In [457]:
coeff = ((v*a**2*y**2 - v*a**2 - w**2*a**2*y**2 + w**2 - I*w*a**2*y**2 + I*w*a**2) / norm).simplify()
coeff

(V_0*a**2*y**2 - V_0*a**2 - \omega**2*a**2*y**2 + \omega**2 - I*\omega*a**2*y**2 + I*\omega*a**2)/(a**2*(y**4 - 2*y**2 + 1))

In [454]:
coeff.subs(v,S(1)/2).subs(a,1).simplify()

(-\omega**2 - I*\omega + 1/2)/(y**2 - 1)

In [461]:
( (I*w - v)/(1-y**2) + (w**2*(1-a**2*y**2))/(a**2*(1-y**2)**2) ).subs(v, S(1)/2).subs(a,1).simplify()

(-\omega**2 - I*\omega + 1/2)/(y**2 - 1)

CORRECT RESULT! YAY!

Find maximum of potential

$V(x) = \alpha^2 V_0 \text{sech}^2(\alpha x)$

In [463]:
V = a**2*v*sech(a*x)**2
V

V_0*a**2*sech(a*x)**2

In [469]:
V_y = V.subs(x, 1/a*atanh(y))
V_y

V_0*a**2*(1 - y**2)

In [472]:
display(diff(V_y, y, 1))
solve(Eq(diff(V_y, y, 1),0))

-2*V_0*a**2*y

[{V_0: 0}, {y: 0}]